In [1]:
import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
from karstification import calc_karstification_for_HU12
from pandas import read_excel
from pynhd import pynhd
from shapely import box
import os
import string
import warnings

hr = pynhd.NHDPlusHR('huc12')
huc12 = pynhd.WaterData('wbd12', crs='epsg:4326')

def format_filename(s):
    """Take a string and return a valid filename constructed from the string.
Uses a whitelist approach: any characters not present in valid_chars are
removed. Also spaces are replaced with underscores.
 
Note: this method may produce invalid filenames such as ``, `.` or `..`
When I use this method I prepend a date string like '2009_01_15_19_46_32_'
and append a file extension like '.txt', so I avoid the potential of using
an invalid filename.
 
"""
    valid_chars = "-_.() %s%s" % (string.ascii_letters, string.digits)
    filename = ''.join(c for c in s if c in valid_chars)
    filename = filename.replace(' ','_') # I don't like spaces in filenames.
    return filename

In [2]:
box_df = read_excel('bounding_boxes.xlsx')
bbox_zip = zip(box_df.x_min, box_df.y_min, box_df.x_max, box_df.y_max)


In [7]:
box_df.iloc[0]

Name                                    East of St. Louis #1, IL
x_min                                                   -90.2594
x_max                                                   -90.1445
y_min                                                    38.3366
y_max                                                    38.4785
rock_type_1                                            limestone
rock_type_2                                            sandstone
rock_type_3     dolostone (dolomite); sedimentary breccia; chert
unit_name      Upper Valmeyeran (Aux Vases,Ste.Genevieve,St.L...
unit_age                                           Mississippian
Name: 0, dtype: object

In [3]:
for i, bbox in enumerate(bbox_zip):
    if i==0:
        boxname = box_df.Name[i]

        box_hucs12 = huc12.bybox(bbox)
        box_poly = box(*bbox)
        box_gdf = gpd.GeoDataFrame(index=[0], crs='epsg:4326', geometry=[box_poly])
        box_dirname = format_filename(boxname)
        box_path = os.path.join('./qgis/', box_dirname)
        if not os.path.exists(box_path):
            os.makedirs(box_path)
        box_gdf.to_file(os.path.join(box_path, box_dirname + '.shp'))
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            box_hucs12.to_file(os.path.join(box_path, 'box_hucs.shp'))

        
        


In [18]:
hu = box_hucs12.iloc[0]

In [17]:
calc_karstification_for_HU12(hu, box_dirname)

ReadTimeout: HTTPSConnectionPool(host='elevation.nationalmap.gov', port=443): Read timed out. (read timeout=30)

In [4]:
import py3dep

In [5]:
hu = box_hucs12.iloc[4]
this_hu12 = hu.geometry
dem = py3dep.get_map("DEM", this_hu12, resolution=5)


ValueError: 'driver' is required to read/write dataset.

In [8]:
bbox

(-90.728075, 43.056946, -90.6389211, 43.1225278)

In [20]:
#bbox = (-92.106649, -91.982017,	43.504401, 43.593743) # MN
bbox = (-90.259400,	-90.144500,	38.336600,	38.478500) # St L 1

avail = py3dep.check_3dep_availability(bbox)

In [21]:
avail

{'1m': True,
 '3m': True,
 '5m': False,
 '10m': True,
 '30m': True,
 '60m': False,
 'topobathy': True}

In [17]:
box_df

,Name,x_min,x_max,y_min,y_max,rock_type_1,rock_type_2,rock_type_3,unit_name,unit_age
0,"East of St. Louis #1, IL",-90.259400,-90.144500,38.336600,38.478500,limestone,sandstone,dolostone (dolomite); sedimentary breccia; chert,"Upper Valmeyeran (Aux Vases,Ste.Genevieve,St.L...",Mississippian
1,"East of St. Louis #2, IL",-90.243000,-90.128600,38.192800,38.332500,limestone,sandstone,dolostone (dolomite); sedimentary breccia; chert,NaN,NaN
2,Meramecian-east-southeast Missouri,-90.113000,-89.998800,37.873300,38.012000,limestone,NaN,NaN,Meramecian Series,Late Mississippian-Meramecian Series
3,Meramecian-southwest Missouri,-94.308000,-94.157000,37.051000,37.229000,limestone,NaN,NaN,Meramecian Series,Late Mississippian-Meramecian Series
4,"Jo Daviess #1, IL",-90.434406,-90.345096,42.439889,42.506144,limestone,dolostone (dolomite),shale,Galena Grp,Ordovician
5,"Jo Daviess #2, IL",-90.206870,-90.117560,42.428986,42.495217,limestone,dolostone (dolomite),shale,Galena Grp,Ordovician
6,"Jo Daviess #3, IL",-90.355473,-90.266163,42.207148,42.273382,limestone,dolostone (dolomite),shale,Galena Grp,Ordovician
7,south_dakota_one,-103.980500,-103.852100,44.154500,44.241600,limestone,dolomite,chert,"Madison_grp, Paha Sapa Lm.",Mississippian
8,nw_montana_1,-114.216700,-114.116700,47.919800,48.010800,carbonate,meta-argillite,quartzite,Piegan_grp,Mesoproterozoic
9,nw_montana_2,-114.566600,-114.431200,48.316300,48.406500,meta-argillite,dolomite,quartzite,Wallace_fm,Mesoproterozoic
